# 차선검출

## <br>라이브러리

In [47]:
import cv2
import numpy as np
from math import *

## <br>함수부

In [48]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices, color3=(255,255,255), color1=255):    # ROI 설정
    mask = np.zeros_like(img)
    
    if len(img.shape) > 2:
        color = color3
    else:
        color = color1
        
    cv2.fillPoly(mask, vertices, color)    # ROI부분을 color로 채움
    
    ROI_image = cv2.bitwise_and(img, mask)
    return ROI_image

def draw_lines(img, lines, color=[0, 0, 255], thickness=2):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def draw_fit_line(img, lines, color=[255, 0, 0], thickness=10): # 대표선 그리기
    cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):    # 허프변환
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    # line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    # draw_lines(line_img, lines)
    
    return lines

def weighted_img(img, initial_img, a=1, b=1., c=0.):    # 두 이미지 겹치기
    return cv2.addWeighted(initial_img, a, img, b, c)

def get_fitline(img, f_lines): # 대표선 구하기
    lines = np.squeeze(f_lines)
    lines = lines.reshape(lines.shape[0]*2,2)
    rows,cols = img.shape[:2]
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]-1)-y)/vy*vx+x), img.shape[0]-1
    x2, y2 = int(((img.shape[0]/2+100)-y)/vy*vx+x), int(img.shape[0]/2+100)
    
    result = [x1,y1,x2,y2]
    return result

def get_midline(left_fit_line, right_fit_line):
    midline = [int((left_fit_line[0]+right_fit_line[0])/2),int((left_fit_line[1]+right_fit_line[1])/2),int((left_fit_line[2]+right_fit_line[2])/2),int((left_fit_line[3]+right_fit_line[3])/2)]
    
    return midline

def get_midline_degree(left_fit_line, right_fit_line):    # 중간라인 기울기 구하기. 아랫점이 왼쪽일때 기울기 +.
    midline = [int((left_fit_line[0]+right_fit_line[0])/2),int((left_fit_line[1]+right_fit_line[1])/2),int((left_fit_line[2]+right_fit_line[2])/2),int((left_fit_line[3]+right_fit_line[3])/2)]
    degree = degrees(atan((midline[3]-midline[1])/(midline[2]-midline[0])))
    
    return degree

## <br>이미지 처리 (grayscale - invert - blur - canny)

In [49]:
# original_image = cv2.imread('C:/Users/admin/Desktop/Hustar/Project/linedetect/source.jpg')
original_image = cv2.imread('C:/Users/admin/Downloads/roaddata/Training/bb/data/front/20200708_200952/1/1_20200708_200952_000000.jpg')
original_image = cv2.imread('C:/Users/admin/Downloads/solidWhiteCurve.jpg')
gray_image = grayscale(original_image)
invert_image = cv2.bitwise_not(gray_image)
blur_image = gaussian_blur(invert_image, 5)
canny_image = canny(blur_image, 70, 100)

original_image_resized = cv2.resize(original_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
gray_image_resized = cv2.resize(gray_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
invert_image_resized = cv2.resize(invert_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
blur_image_resized = cv2.resize(blur_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
canny_image_resized = cv2.resize(canny_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
cv2.imshow('original', original_image_resized)
cv2.imshow('gray', gray_image_resized)
cv2.imshow('invert', invert_image_resized)
cv2.imshow('blur', blur_image_resized)
cv2.imshow('canny', canny_image_resized)

cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>ROI 위치 설정

invert test용 라인 (ROI 위치 확인용)

In [50]:
height, width = original_image.shape[:2]
vertices = np.array([[(width/4-60, height), (width/2-45, height/3+45), (width/2+45, height/3+45), (width*3/4+60, height)]], dtype=np.int32)
# vertices = np.array([[(width/4-60, height), (width/2-45, height-200), (width/2+45, height-200), (width*3/4+60, height)]], dtype=np.int32)

roi_image = region_of_interest(invert_image, vertices)
roi_resized = cv2.resize(roi_image, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
cv2.imshow('ROI', roi_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

canny 실전용 라인

In [51]:
height, width = original_image.shape[:2]
# 여기에 vertice 라인 넣음. 추후에
roi_image = region_of_interest(canny_image, vertices)
cv2.imshow('ROI', roi_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <br>HoughLine 적용 및 기울기 제한

In [55]:
line_arr = hough_lines(roi_image, 1, 1*np.pi/180, 30, 65, 20)    # 허프 변환
line_arr = np.squeeze(line_arr)

# 기울기 구하기
slope_degree = (np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:,2]) * 180) / np.pi

# 수평 기울기 제한
line_arr = line_arr[np.abs(slope_degree)<160]
slope_degree = slope_degree[np.abs(slope_degree)<160]
# 수직 기울기 제한
line_arr = line_arr[np.abs(slope_degree)>95]
slope_degree = slope_degree[np.abs(slope_degree)>95]
# 필터링된 직선 버리기
L_lines, R_lines = line_arr[(slope_degree>0),:], line_arr[(slope_degree<0),:]
temp = np.zeros((original_image.shape[0], original_image.shape[1], 3), dtype=np.uint8)
L_lines, R_lines = L_lines[:,None], R_lines[:,None]
# 왼쪽, 오른쪽 각각 대표선 구하기
left_fit_line = get_fitline(roi_image, L_lines)
right_fit_line = get_fitline(roi_image, R_lines)
# 가운데 대표선 구하기
mid_fit_line = get_midline(left_fit_line, right_fit_line)
# 대표선 그리기
draw_fit_line(temp, left_fit_line)
draw_fit_line(temp, right_fit_line)
# 가운데 라인 그리기
draw_fit_line(temp, mid_fit_line)

result = weighted_img(temp, original_image)
result_resized = cv2.resize(result, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
Hough_resized = cv2.resize(temp, dsize=(0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
cv2.imshow('HoughLine', Hough_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow('result', result_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [44]:
mid_fit_line

[530, 539, 495, 370]

In [45]:
get_midline_degree(left_fit_line, right_fit_line)

78.29942053718419

In [46]:
line_arr

array([[490, 313, 678, 417],
       [483, 311, 693, 432],
       [408, 366, 535, 274],
       [484, 309, 683, 419],
       [280, 462, 345, 410],
       [538, 342, 691, 430],
       [420, 356, 496, 301]], dtype=int32)

생각

mid_fit_line[0] 과 mid_fit_line[2] 기준.
ex) 977 - 933 = +값 나옴. 이는 왼쪽으로 좀더 치우쳤단 소리이므로 왼쪽바퀴 좀더 가속해서 오른쪽으로 더 가게 함.
-값 나오면 오른쪽으로 좀더 치우쳤단 소리이므로 오른쪽바퀴 좀더 가속해서 왼쪽으로 더 가게 함.